# Greenhouse gas emissions to concentrations

Should be relatively straightforward, particularly for non-CO2 and non-CH4. FaIR 2.0 results should reproduce.

Also, RCMIP's emissions and concentrations are inconsistent. CFC11 has emissions starting in 1943, but no delta concentrations until 1945. (unless there's some kind of 3 year delay going on for stratospheric mixing, but that's only relevant for ozone). So this is at best a sense check. See also my point about natural emissions of F-gases.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl
from scipy.interpolate import interp1d
import pickle
from climateforcing.utils import mkdir_p

from fair21.constants.gases import molwt, burden_per_emission, lifetime, gas_list
from fair21.defaults.gases import (
    partition_fraction,
    pre_industrial_concentration,
    natural_emissions_adjustment,
    iirf_0,
    iirf_cumulative,
    iirf_temperature,
    iirf_airborne,
    iirf_horizon
)
from fair21.gas_cycle import (
    calculate_g,
    calculate_alpha,
    step_concentration_1box,
    step_concentration_co2
)

In [ ]:
burden_per_emission['CO2']

In [ ]:
# grab some emissions
emissions = {}
df = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
for gas in gas_list:
    gas_rcmip_name = gas.replace("-", "")
    emissions[gas] = df.loc[
        (df['Scenario']=='ssp245') & (df['Variable'].str.endswith("|"+gas_rcmip_name)) & (df['Region']=='World'), '1750':
    ].interpolate(axis=1).values.squeeze()
    
# CO2 and N2O units need to behave
emissions["CO2"] = emissions["CO2"] / 1000
emissions["N2O"] = emissions["N2O"] / 1000
#co2_emissions = 1/1000 * df.loc[(df['Scenario']=='ssp245') & (df['Variable']=='Emissions|CO2') & (df['Region']=='World'), '1750':].interpolate(axis=1).values.squeeze()
#ch4_emissions = df.loc[(df['Scenario']=='ssp245') & (df['Variable']=='Emissions|CH4') & (df['Region']=='World'), '1750':].interpolate(axis=1).values.squeeze()
#cfc11_emissions = df.loc[(df['Scenario']=='ssp245') & (df['Variable']=='Emissions|Montreal Gases|CFC|CFC11') & (df['Region']=='World'), '1750':].interpolate(axis=1).values.squeeze()

In [ ]:
# initialise dicts for outputs. These will eventually be moved to the forward model
concentration = {}
airborne_emissions = {}
cumulative_emissions = {}

In [ ]:
# grab some concentrations
concentration_rcmip = {}
df = pd.read_csv('../data/rcmip/rcmip-concentrations-annual-means-v5-1-0.csv')
for gas in gas_list:
    gas_rcmip_name = gas.replace("-", "")
    concentration_rcmip[gas] = df.loc[
        (df['Scenario']=='ssp245') & (df['Variable'].str.endswith("|"+gas_rcmip_name)) & (df['Region']=='World'), '1750':
    ].interpolate(axis=1).values.squeeze()

In [ ]:
# grab indicative temperature projections
df = pd.read_csv('../data/rcmip-phase2/rcmip-phase2-fair162-ssp245-mean-temperature.csv')
ssp245_temperature_rfmip = df['temperature'].values

## CFC11

inocuous test case (but ozone depleting... so should pass this feedback on to ozone forcing)

In [ ]:
emissions['CFC-11']

In [ ]:
burden_per_emission['CFC-11']  # ppt increase per kt CFC-11 emission

In [ ]:
n_timesteps = len(emissions['CFC-11'])
concentration['CFC-11'] = np.ones(n_timesteps) * np.nan

airborne_emissions = 0
#concentration['CFC-11'][0] = pre_industrial_concentration['CFC-11']

for i in range(n_timesteps):
    concentration['CFC-11'][i], airborne_emissions = step_concentration_1box(
        emissions['CFC-11'][i], 
        airborne_emissions,
        lifetime['CFC-11'],
        burden_per_emission['CFC-11'],
        pre_industrial_concentration=pre_industrial_concentration['CFC-11'],
        timestep=1
    )

In [ ]:
# try a monthly timestep
t_new = np.arange(1750+1/24, 2501, 1/12)
f = interp1d(np.arange(1750.5, 2501), emissions['CFC-11'], fill_value="extrapolate")
cfc11_emissions_monthly = f(t_new)

In [ ]:
cfc11_concentrations_monthly = np.ones(len(cfc11_emissions_monthly)) * np.nan

airborne_emissions = 0
cfc11_concentrations_monthly[0] = pre_industrial_concentration['CFC-11']

for i in range(len(cfc11_concentrations_monthly)):
    cfc11_concentrations_monthly[i], airborne_emissions = step_concentration_1box(
        cfc11_emissions_monthly[i], 
        airborne_emissions,
        lifetime['CFC-11'],
        burden_per_emission['CFC-11'],
        pre_industrial_concentration=pre_industrial_concentration['CFC-11'],
        timestep=1/12
    )

In [ ]:
pl.plot(np.arange(1750.5, 2501), concentration['CFC-11'])
pl.plot(np.arange(1750.5, 2501), concentration_rcmip['CFC-11'])
pl.plot(np.arange(1750+1/24, 2501, 1/12), cfc11_concentrations_monthly)

## CO2

important to get correct

In [ ]:
emissions['CO2']  # unit: GtCO2 / yr

In [ ]:
concentration['CO2'] = np.ones(n_timesteps) * np.nan

co2_boxes = 0
airborne_emissions = 0
cumulative_emissions['CO2'] = np.cumsum(emissions['CO2'])

# g1 = np.sum(partition_fraction['CO2'] * lifetime['CO2'] * (1 - (1 + iirf_horizon/lifetime['CO2']) * np.exp(-iirf_horizon/lifetime['CO2'])))
# g0 = 1/(np.sinh(np.sum(partition_fraction['CO2']*lifetime['CO2']*(1 - np.exp(-iirf_horizon/lifetime['CO2'])) , axis=-1)/g1))
g0, g1 = calculate_g(lifetime['CO2'], partition_fraction=partition_fraction['CO2'])

print(g1, g0)

for i in range(n_timesteps):
    alpha_lifetime = calculate_alpha(
        cumulative_emissions['CO2'][i],
        airborne_emissions,
        ssp245_temperature_rfmip[i],
        iirf_0["CO2"],
        iirf_cumulative["CO2"],
        iirf_temperature["CO2"],
        iirf_airborne["CO2"],
        g0,
        g1,
    )
    concentration['CO2'][i], co2_boxes, airborne_emissions = step_concentration_co2(
        emissions['CO2'][i], 
        co2_boxes,
        airborne_emissions, 
        burden_per_emission['CO2'],
        alpha_lifetime=alpha_lifetime,
        pre_industrial_concentration=pre_industrial_concentration['CO2'],
        timestep=1,
    )

Here, we see an almost perfect agreement with the MAGICC-derived CO2 projections with appropriate carbon cycle parameter tuning

In [ ]:
pl.plot(np.arange(1750.5, 2501), concentration['CO2'], label='FaIR 2.1')
pl.plot(np.arange(1750.5, 2501), concentration_rcmip['CO2'], label='History + MAGICC6')
pl.legend()
pl.title('CO2 concentrations: SSP2-4.5')

In [ ]:
concentration['CO2'][270], concentration_rcmip['CO2'][270]

## Methane: an in-between case

methane has a lifetime feedback, and it also affects ozone and stratospheric water vapor forcing.

The step concentration for a one-box gas will work here, but we need an alpha adjustment.

In [ ]:
concentration['CH4'] = np.ones(n_timesteps) * np.nan

ch4_boxes = 0
airborne_emissions = 0
concentration['CH4'][0] = pre_industrial_concentration['CH4']
cumulative_emissions['CH4'] = np.cumsum(emissions['CH4']) # cumulative emissions are not contributing to methane lifetime, but we'll track them in case future research makes them relevant

g0, g1 = calculate_g(lifetime['CH4'])

ch4_lifetime = np.ones(n_timesteps) * np.nan

for i in range(n_timesteps):
    alpha_lifetime = calculate_alpha(
        cumulative_emissions['CH4'][i],
        airborne_emissions,
        ssp245_temperature_rfmip[i],
        iirf_0["CH4"],
        iirf_cumulative["CH4"],
        iirf_temperature["CH4"],
        iirf_airborne["CH4"],
        g0,
        g1,
    )
    
    ch4_lifetime[i] = alpha_lifetime * lifetime['CH4']
    
    concentration['CH4'][i], airborne_emissions = step_concentration_1box(
        emissions['CH4'][i], 
        airborne_emissions, 
        lifetime['CH4'],
        burden_per_emission['CH4'],
        alpha_lifetime=alpha_lifetime,
        pre_industrial_concentration=pre_industrial_concentration['CH4'],
        timestep=1,
    )

In [ ]:
pl.plot(np.arange(1750.5, 2501), concentration['CH4'], label='FaIR 2.1')
pl.plot(np.arange(1750.5, 2501), concentration_rcmip['CH4'], label='History + MAGICC6')
pl.legend()
pl.title('CH4 concentrations: SSP2-4.5')
pl.plot(2020, 1870, 'k+')

In [ ]:
pl.plot(np.arange(1750.5, 2501), ch4_lifetime)

In [ ]:
concentration['CH4'][270]

In [ ]:
ch4_lifetime[258:268].mean()  # good job, Nick. IPCC assess 9.1

## Make plot similar to Nick's from FaIR 2.0.0-alpha

Some of the minor GHGs where natural emissions are not zero have misbehaviour after 2100. This is the fault of the scenario, not FaIR.

In [ ]:
for gas in gas_list:
    if gas in ['CH4', 'CO2']:
        continue  # done above and slightly different to rest!
    
    concentration[gas] = np.ones(n_timesteps) * np.nan

    airborne_emissions = 0

    for i in range(n_timesteps):
        concentration[gas][i], airborne_emissions = step_concentration_1box(
            emissions[gas][i], 
            airborne_emissions,
            lifetime[gas],
            burden_per_emission[gas],
            pre_industrial_concentration=pre_industrial_concentration[gas],
            timestep=1,
            natural_emissions_adjustment=natural_emissions_adjustment[gas]
        )

In [ ]:
fig, ax = pl.subplots(6, 8, figsize=(16,16))
for igas, gas in enumerate(gas_list):
    iy = igas % 8
    ix = igas // 8
    ax[ix, iy].plot(np.arange(1750.5, 2501), concentration[gas], label='FaIR 2.1')
    ax[ix, iy].plot(np.arange(1750.5, 2501), concentration_rcmip[gas], label='History + MAGICC6')
    ax[ix, iy].set_title(gas)
fig.tight_layout()

In [ ]:
concentration['HFC-227ea'][269]

In [ ]:
# would be awesome to now convert this to csv
mkdir_p("../data/output/")
with open("../data/output/rcmip-fair21-ssp245-concentrations.pkl","wb") as fileout:
    pickle.dump(concentration, fileout)